In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torchvision import datasets, transforms
from torch import nn
from torch import optim
import torch.nn.functional as F

In [2]:
data=pd.read_csv("train.csv")
df=data.drop(columns=["id"])

# sample = np.random.choice(data.index, size=20)
# df = df.iloc[sample]

y=df['target']
y=y.values.reshape(200000,1)
# x=sample_df.drop('target', axis=1)
for j in range(200000):
    i=y[j]
    if i=='Class_1':
        y[j]=0
    if i=='Class_2':
        y[j]=1
    if i=='Class_3':
        y[j]=2
    if i=='Class_4':
        y[j]=3
    if i=='Class_5':
        y[j]=4
    if i=='Class_6':
        y[j]=5
    if i=='Class_7':
        y[j]=6
    if i=='Class_8':
        y[j]=7
    if i=='Class_9':
        y[j]=8


#0,1,4,6,7,26,30,40,44,54,60
# 15,16,24,36,46,47,55,73
df=df.drop(columns=["feature_0","feature_1","feature_4","feature_6","feature_7","feature_26","feature_30","feature_40","feature_44","feature_54","feature_60","feature_15","feature_16","feature_24","feature_36","feature_46","feature_47","feature_55","feature_73"])
df=df.astype(float)

df=torch.tensor(df.values)
q1,q2=torch.max(df,dim=0)
df=torch.divide(df,q1)
# df[:,:75]=df[:,:75].float()
# df[:,75]=df[:,75]*8
# df[:,75]=df[:,75].long()
df[:,:56]=df[:,:56].float()
df[:,56]=df[:,56]*8
df[:,56]=df[:,56].long()


In [3]:
dataloader = torch.utils.data.DataLoader(df,batch_size=64,shuffle=True)

In [ ]:
tdf=pd.read_csv("train.csv")
tdf=tdf.drop(columns=["id"])

# taking a 1000 sample to test training 
sample = np.random.choice(data.index, size=2000)
sample_tdf = tdf.iloc[sample]

yt=sample_tdf['target']
yt=yt.values.reshape(2000,1)
# x=sample_df.drop('target', axis=1)
for j in range(2000):
    i=yt[j]
    if i=='Class_1':
        yt[j]=0
    if i=='Class_2':
        yt[j]=1
    if i=='Class_3':
        yt[j]=2
    if i=='Class_4':
        yt[j]=3
    if i=='Class_5':
        yt[j]=4
    if i=='Class_6':
        yt[j]=5
    if i=='Class_7':
        yt[j]=6
    if i=='Class_8':
        yt[j]=7
    if i=='Class_9':
        yt[j]=8

#sample_tdf = sample_tdf.apply(pd.to_numeric,downcast='float')
sample_tdf=sample_tdf.astype(int)
sample_tdf=torch.tensor(sample_tdf.values)
q1,q2=torch.max(sample_tdf,dim=0)
sample_tdf=torch.divide(sample_tdf,q1)
sample_tdf[:,:75]=sample_tdf[:,:75].float()
sample_tdf[:,75]=sample_tdf[:,75]*8
sample_tdf[:,75]=sample_tdf[:,75].long()
tdataloader = torch.utils.data.DataLoader(sample_tdf,batch_size=64,shuffle=True)

In [5]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1=nn.Linear(56,40)
        self.fc2=nn.Linear(40,30)
        self.fc3=nn.Linear(30,20)
        self.fc4=nn.Linear(20,9)
        #self.dropout=nn.Dropout(p=0.2)
    
    def forward(self,x):
        x=x.view(x.shape[0],-1)
        # x=self.dropout(F.relu(self.fc1(x)))
        # x=self.dropout(F.relu(self.fc2(x)))
        # x=self.dropout(F.relu(self.fc3(x)))
        # x=F.log_softmax(self.fc4(x),dim=1)
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=F.relu(self.fc3(x))
        x=F.log_softmax(self.fc4(x),dim=1)
        return x



In [6]:
model=Network()
criterion=nn.NLLLoss()
optimizer=optim.SGD(model.parameters(),lr=0.6)
epoch=30

for e in range(epoch):
    run_loss=0
    for data in dataloader:
        x=data[:,:56].float()
        y=data[:,56].long()
        #print(y)
        optimizer.zero_grad()
        out=model(x)
        #print(torch.exp(out))
        loss=criterion(out,y)
        loss.backward()
        optimizer.step()
        run_loss+=loss.item()
    else :
        with torch.no_grad():
            model.eval()
            ps=model(x)
            model.train()
        p=torch.exp(ps)
        top_val,top_indx=p.topk(1,dim=1)
        #print(top_indx,y)
        match=top_indx==y.view(*top_indx.shape)
        acc=torch.mean(match.type(torch.FloatTensor))
        print("epoch ",e+1,"  Train Accuracy =",acc.item()*100,"  Train Loss =",run_loss/len(dataloader))

# model=Network()
# criterion=nn.NLLLoss()
# optimizer=optim.SGD(model.parameters(),lr=0.6)
# epoch=20

# for e in range(epoch):
#     run_loss=0
#     for data in dataloader:
#         x=data[:,:75].float()
#         y=data[:,75].long()
#         #print(y)
#         optimizer.zero_grad()
#         out=model(x)
#         #print(torch.exp(out))
#         loss=criterion(out,y)
#         loss.backward()
#         optimizer.step()
#         run_loss+=loss.item()
#     else :
#         with torch.no_grad():
#             model.eval()
#             ps=model(x)
#             model.train()
#         p=torch.exp(ps)
#         top_val,top_indx=p.topk(1,dim=1)
#         #print(top_indx,y)
#         match=top_indx==y.view(*top_indx.shape)
#         acc=torch.mean(match.type(torch.FloatTensor))

#         run_loss1=0
#         for tdata in tdataloader:
#                 xt=tdata[:,:75]
#                 yt=tdata[:,75]
#                 yt = yt.type(torch.LongTensor)
#                 with torch.no_grad():
#                     model.eval()
#                     ps=model(xt)
#                     model.train()
#                 p=torch.exp(ps)
#                 max_val,max_indx=p.topk(1,dim=1)
#                 match=max_indx==yt.view(*max_indx.shape)
#                 acc1=torch.mean(match.type(torch.FloatTensor))
#                 run_loss1+=acc1.item()
#         print("epoch ",e+1,"  Train Accuracy =",acc.item()*100," Test accuracy =",run_loss1*100/len(tdataloader),"  Train Loss =",run_loss/len(dataloader))

epoch  1   Train Accuracy = 34.375   Train Loss = 1.8399406179046631
epoch  2   Train Accuracy = 43.75   Train Loss = 1.8077767453002929
epoch  3   Train Accuracy = 29.6875   Train Loss = 1.8017017454528808
epoch  4   Train Accuracy = 34.375   Train Loss = 1.79614563331604
epoch  5   Train Accuracy = 34.375   Train Loss = 1.7923634576034546
epoch  6   Train Accuracy = 28.125   Train Loss = 1.7903159042358399
epoch  7   Train Accuracy = 35.9375   Train Loss = 1.7886287504959106
epoch  8   Train Accuracy = 31.25   Train Loss = 1.787098802947998
epoch  9   Train Accuracy = 42.1875   Train Loss = 1.7858175561141967
epoch  10   Train Accuracy = 39.0625   Train Loss = 1.7844912972640992
epoch  11   Train Accuracy = 32.8125   Train Loss = 1.7837494951629638
epoch  12   Train Accuracy = 42.1875   Train Loss = 1.7830065526199341
epoch  13   Train Accuracy = 35.9375   Train Loss = 1.7821045766830443
epoch  14   Train Accuracy = 29.6875   Train Loss = 1.7809850686264037
epoch  15   Train Accuracy

In [ ]:
remove=[0,1,4,6,7,26,30,40,44,54,60,15,16,24,36,46,47,55,73]

In [ ]:
# for i in range(56):
#     if i>=0:
#         class Network(nn.Module):
#             def __init__(self):
#                 super().__init__()
#                 self.fc1=nn.Linear(55,40)
#                 self.fc2=nn.Linear(40,25)
#                 self.fc3=nn.Linear(25,9)
#                 #self.dropout=nn.Dropout(p=0)
            
#             def forward(self,x):
#                 x=x.view(x.shape[0],-1)
#                 # x=self.dropout(F.relu(self.fc1(x)))
#                 # x=self.dropout(F.relu(self.fc2(x)))
#                 # x=self.dropout(F.relu(self.fc3(x)))
#                 # x=self.dropout(F.relu(self.fc4(x)))
#                 # x=F.log_softmax(self.fc5(x),dim=1)
#                 x=F.relu(self.fc1(x))
#                 x=F.relu(self.fc2(x))
#                 x=F.log_softmax(self.fc3(x),dim=1)
#                 return x


#         model=Network()
#         criterion=nn.NLLLoss()
#         optimizer=optim.SGD(model.parameters(),lr=0.6)
#         epoch=1


#         for e in range(epoch):
#             run_loss=0
#             for data in dataloader:
#                 x=data[:,:56]
#                 x = torch.cat((x[:,:i],x[:,i+1:]),dim=1)
#                 x=x.float()
#                 y=data[:,56].long()
#                 #print(y)
#                 optimizer.zero_grad()
#                 out=model(x)
#                 #print(torch.exp(out))
#                 loss=criterion(out,y)
#                 loss.backward()
#                 optimizer.step()
#                 run_loss+=loss.item()
#             else :
#                 with torch.no_grad():
#                     model.eval()
#                     ps=model(x)
#                     model.train()
#                 p=torch.exp(ps)
#                 top_val,top_indx=p.topk(1,dim=1)
#                 #print(top_indx,y)
#                 match=top_indx==y.view(*top_indx.shape)
#                 acc=torch.mean(match.type(torch.FloatTensor))
#                 print("col ",i,"  Train Accuracy =",acc.item()*100,"  Train Loss =",run_loss/len(dataloader))

In [ ]:
checkpoint = {'input_size': 56,
              'output_size': 9,
              'hidden_layers': [40,30,20],
              'state_dict': model.state_dict()}

torch.save(checkpoint, 'checkpoint.pth')

In [3]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = Network()
    model.load_state_dict(checkpoint['state_dict'])
    return model

model = load_checkpoint('checkpoint.pth')
print(model)

Network(
  (fc1): Linear(in_features=56, out_features=40, bias=True)
  (fc2): Linear(in_features=40, out_features=30, bias=True)
  (fc3): Linear(in_features=30, out_features=20, bias=True)
  (fc4): Linear(in_features=20, out_features=9, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [7]:
df1=pd.read_csv("test.csv")
dfy=df1["id"]
dfx=df1.drop(columns=["id","feature_0","feature_1","feature_4","feature_6","feature_7","feature_26","feature_30","feature_40","feature_44","feature_54","feature_60","feature_15","feature_16","feature_24","feature_36","feature_46","feature_47","feature_55","feature_73"])

dfx = dfx.astype(float)
dfx=torch.tensor(dfx.values)

dfy=dfy.values.reshape(len(df1),1)
dfy=torch.from_numpy(dfy)

q1,q2=torch.max(dfx,dim=0)
dfx=torch.divide(dfx,q1)
df1=torch.cat((dfy,dfx),dim=1)

testloader = torch.utils.data.DataLoader(df1,batch_size=100000,shuffle=True)

In [9]:
model.eval()
pridict=pd.DataFrame()
ids=pd.DataFrame()
for data in testloader:
    id=data[:,0]
    x=data[:,1:57]
    with torch.no_grad():
        ps=model(x.float())
    p=torch.exp(ps)
    #max_val,max_indx=p.topk(1,dim=1)
    
    #max_indx=max_indx.squeeze().tolist()
    #max_indx=max_indx[:]
    #id=id.squeeze().tolist()
    #id=id[:]
    p=p.float()
    id=id.long()
    df_p=pd.DataFrame(p,columns=["Class_1","Class_2","Class_3","Class_4","Class_5","Class_6","Class_7","Class_8","Class_9"])
    pridict=pridict.append(df_p)
    df_id=pd.DataFrame(id,columns=["id"])
    ids=ids.append(df_id)

In [10]:
pridict=pridict.astype(float)
ids=ids.astype(int)
df=pd.concat([ids,pridict],axis=1)
df.to_csv('final_pridictions_2.csv')